In [1]:
import pandas as pd
import csv
import numpy as np

The below data is the data used in the illustration for the proposed method provided in the paper. This data has collected from various sensors based on some experiment conducted by the authors of the paper.

In [14]:
sensor_data=pd.read_csv('sensor_data.csv')
sensor_data

,Time(s),m(A),m(B),m(C),m(AB),m(AC),m(BC),m(theta)
0,1,0.30,0.00,0.00,0.50,0.0,0.2,0.0
1,3,0.00,0.25,0.00,0.15,0.4,0.2,0.0
2,4,0.60,0.20,0.00,0.10,0.1,0.0,0.0
3,6,0.55,0.15,0.00,0.10,0.1,0.0,0.1
4,10,0.75,0.00,0.00,0.15,0.1,0.0,0.0
5,30,0.25,0.65,0.00,0.00,0.0,0.0,0.1
6,40,0.00,0.00,0.00,0.00,0.0,0.0,1.0
7,50,0.60,0.30,0.00,0.00,0.0,0.0,0.1
8,80,0.70,0.20,0.00,0.10,0.0,0.0,0.0
9,110,0.60,0.20,0.00,0.00,0.2,0.0,0.0


Taking t=10s as an example to illustrate the whole process

In [17]:
from visibility_graph import visibility_graph

In [37]:
A=sensor_data['m(A)']
B=sensor_data['m(B)']
C=sensor_data['m(C)']
AB=sensor_data['m(AB)']
AC=sensor_data['m(AC)']
BC=sensor_data['m(BC)']
theta=sensor_data['m(theta)']

The below cell describes the implementation of the VGA operator derived from the visibility graph.

In [72]:
def vga(vis_graph):
    node_degree={}
    sum_degree=0
    vga_val=0
    for i in range(0,5):
        count=0
        for v in vis_graph.edges():
            if v[0]==i or v[1]==i:
                count=count+1
        node_degree[i]=count
        sum_degree+=count

    for i in range(0,5):
        vga_val+=vis_graph.node[i]['mag']*(node_degree[i])/(sum_degree)
    
    return vga_val

In [41]:
a=A[0:5]
b=B[0:5]
c=C[0:5]
ab=AB[0:5]
ac=AC[0:5]
bc=BC[0:5]
theta1=theta[0:5]

In [76]:
srpr=[]

a_vis=visibility_graph(a)
srprA=vga(a_vis)
srpr.append(srprA)

b_vis=visibility_graph(b)
srprB=vga(b_vis)
srpr.append(srprB)

c_vis=visibility_graph(c)
srprC=vga(c_vis)
srpr.append(srprC)

ab_vis=visibility_graph(ab)
srprAB=vga(ab_vis)
srpr.append(srprAB)

ac_vis=visibility_graph(ac)
srprAC=vga(ac_vis)
srpr.append(srprAC)

bc_vis=visibility_graph(bc)
srprBC=vga(bc_vis)
srpr.append(srprBC)

theta_vis=visibility_graph(theta1)
srprTheta=vga(theta_vis)
srpr.append(srprTheta)

In [80]:
srpr

[0.4666666666666667,
 0.15999999999999998,
 0.0,
 0.19999999999999998,
 0.18,
 0.08000000000000002,
 0.030000000000000006]

Calculating SRBB:

In [81]:
sum_srpr=0
for i in srpr:
    sum_srpr+=i
    
m_sr=[]
for i in srpr:
    m_sr.append(i/sum_srpr)

In [82]:
m_sr

[0.417910447761194,
 0.14328358208955222,
 0.0,
 0.1791044776119403,
 0.16119402985074627,
 0.07164179104477614,
 0.026865671641791048]

In [141]:
#Using m4* and alpa5 from the illustration
m4=[0.786,0.109,0.002,0.036,0.037,0,0.030]
alpha5=0.7662

In [142]:
for i in range(len(m4)):
    m4[i]=m4[i]*alpha5
m4

[0.6022332,
 0.0835158,
 0.0015324,
 0.0275832,
 0.028349399999999997,
 0.0,
 0.022986]

In [119]:
from pyds import MassFunction

In [143]:
m_sr_dict={}
m4_dict={}
temp_list=['A','B','C','AB','AC','BC','ABC']
for i in range(len(temp_list)):
    m_sr_dict[temp_list[i]]=m_sr[i]
    m4_dict[temp_list[i]]=m4[i]
    

In [144]:
m5_sr = MassFunction(m_sr_dict)
m4_prev=MassFunction(m4_dict)

In [145]:
#Combining both BBAs using Dempster Combination
f5=m5_sr&m4_prev

In [146]:
f5

{frozenset({'A'}): 0.8835719047805797,
 frozenset({'B'}): 0.0760853366798698,
 frozenset({'C'}): 0.004164869210557278,
 frozenset({'A', 'B'}): 0.016800658849366643,
 frozenset({'A', 'C'}): 0.015494725283344446,
 frozenset({'B', 'C'}): 0.002823640142750697,
 frozenset({'A', 'B', 'C'}): 0.0010588650535315112}

The above values are the final fusion result values for time node 5 (t=10s)